In [1]:
class Weather():
    
    def __init__(self, name, latitude, longitude, sunrise, sunset, data):
        self.name = name
        self.longitude = longitude
        self.latitude = latitude
        self.sunrise = sunrise
        self.sunset = sunset
        self.data = data
        self.weather = self.data['weather'][0]['main']
        self.description = self.data['weather'][0]['description']
        self.temperature = self.data['main']['temp']
        self.min_temperature = self.data['main']['temp_min']
        self.max_temperature = self.data['main']['temp_max']
        self.pressure = self.data['main']['pressure']
        self.hummidity = self.data['main']['humidity']
        self.sea_level = self.data['main']['sea_level']
        self.grnd_level = self.data['main']['grnd_level']
        self.wind_speed = self.data['wind']['speed']
        self.time = self.data['dt']
        self.time_text = self.data['dt_txt']
        
    def f_temperature(self):
        return round(self.temperature * (9/5) - 459.67, 1)
        
    def f_max_temperature(self):
        return round(self.max_temperature * (9/5) - 459.67, 2)
    
    def f_min_temperature(self):
        return round(self.min_temperature * (9/5) - 459.67, 2)

In [2]:
import requests
import json

wdata = []
count = 0

def getdata_by_cityid(cityid, api_key):
    global count
    count += 1
    base_url = 'http://api.openweathermap.org/data/2.5/forecast?id='
    url = f"{base_url}{cityid}&{api_key}"
    r = requests.get(url).json()
    try:
        name = r['city']['name']
        latitude =  r['city']['coord']['lat']
        longitude = r['city']['coord']['lon']
        sunrise = r['city']['sunrise']
        sunset = r['city']['sunset']
        #print(f"{count}  {name}")
        for i in r['list']:
            wdata.append(Weather(name, latitude, longitude, sunrise, sunset, i))
    except KeyError:
        pass

# List of City ID : http://bulk.openweathermap.org/sample/  
with open('../tmp/current.city.list.json', 'r') as jf: wcity = json.load(jf)

# OpenWeather API KEY : https://openweathermap.org/appid
with open('../tmp/openweathermap.key', 'r') as kf: api_key = kf.read().rstrip()

for city in wcity:
    if city['country'] == 'US':
        getdata_by_cityid(city['id'], api_key)
        
# Takes about 10 min to collect all the data.

In [ ]:
import pandas as pd

dlist = [{'name':i.name, 
          'latitude': i.latitude,
          'longitude': i.longitude, 
          'temperature': i.f_temperature(), 
          'max_temperature': i.f_max_temperature(), 
          'min_temperature': i.f_min_temperature(), 
          'humidity': i.hummidity, 
          'pressure': i.pressure,
          'windspeed': i.wind_speed,
          'time': i.time, 
          'timetxt': i.time_text, 
          'weather': i.weather} for i in wdata]
# print(dlist)

df = pd.DataFrame(dlist)
# print(df)

In [ ]:
import plotly.express as px

# MapBox Token : https://docs.mapbox.com/help/how-mapbox-works/access-tokens/
with open('../tmp/mapbox.key', 'r') as tf: token = tf.read().rstrip()

px.set_mapbox_access_token(token)
fig = px.scatter_mapbox(df, zoom=3, lat="latitude", lon="longitude", 
                        color="weather", 
                        animation_frame="timetxt",
                        #range_color=[80, 20],
                        hover_data=df.columns)
fig.show()

**Output Image**

![Output](weather.jpg "weather")